In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\JaehyunAhn\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import requests
from dataclasses import dataclass
from pprint import pprint
from dotenv import load_dotenv
import os
import time


# .env에서 API 키 불러오기
load_dotenv()
api_key = os.getenv("SERPAPI_KEY")

True

In [6]:
from dataclasses import dataclass

@dataclass
class Paper:
    title: str
    authors: list
    year: str
    snippet: str = None
    journal: str = None
    url: str = None
    cite_count: int = 0

    def __repr__(self):
        return f"Title: {self.title}, Authors: {self.authors}, Year: {self.year}, Journal: {self.journal}, URL: {self.url}"

In [12]:
def fetch_papers(query, as_ylo=10, as_yhi=None, start=0):
    url = "https://serpapi.com/search.json"
    params = {
        "engine": "google_scholar",
        "q": query,
        "as_ylo": as_ylo,
        "as_yhi": as_yhi,
        "api_key": api_key,
        "start": start
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

In [13]:
query = 'source:"Journal of the Association for Information Systems"'

i=0
papers_list = []
while True:
    papers = fetch_papers(query, as_ylo=2022, as_yhi=None, start=i)
    if papers: 
        results = papers.get("organic_results", [])
        if results:
            for result in results:
                title = result.get("title")                
                authors = [author.get("name") for author in result.get("publication_info", {}).get("authors", [])]
                journal = None # 구글 스칼라 검색에서는 직접적으로 저널 명 확인 어려움
                url = result.get("link")
                snippet = result.get("snippet")
                cite_count = result.get("inline_links", {}).get("cited_by", {}).get("total", 0)

                summ = result.get("publication_info", {}).get("summary", "")
                if summ:
                    year = summ.split(",")[-1].strip()[:4]
                else:
                    year = "Unknown"

                paper = Paper(title=title, authors=authors, year=year, journal=journal, url=url, snippet=snippet, cite_count=cite_count)
                papers_list.append(paper)
                print(paper)  # 각 논문 정보 출력
            # 결과 출력
        else:
            print("No results found.")
            break
    i += 10
    time.sleep(2)  # 2초 대기 후 재시도


Title: New Frontiers in information systems theorizing: human-gAI collaboration, Authors: ['S Jarvenpaa', 'S Klein'], Year: 2024, Journal: None, URL: https://aisel.aisnet.org/jais/vol25/iss1/6/
Title: Charting the waters ahead: Moby, Dick, and Jane, Authors: ['DE Leidner'], Year: 2024, Journal: None, URL: https://aisel.aisnet.org/cgi/viewcontent.cgi?article=2218&context=jais
Title: How human resource and information systems practices amplify the returns on information technology investments, Authors: ['A Tafti', 'P Rahmati', 'S Mithas'], Year: 2022, Journal: None, URL: https://aisel.aisnet.org/jais/vol23/iss5/6/
Title: The vicious circles of skill erosion: A case study of cognitive automation, Authors: ['T Rinta-Kahila', 'E Penttinen', 'A Salovaara'], Year: 2023, Journal: None, URL: https://research.aalto.fi/en/publications/the-vicious-circles-of-skill-erosion-a-case-study-of-cognitive-au
Title: Using simulation in information systems research, Authors: ['JQ Dong'], Year: 2022, Journal

In [14]:
import pandas as pd
from dataclasses import asdict

df = pd.DataFrame([asdict(p) for p in papers_list])  # DataFrame으로 변환
df.to_csv("JAIS_papers_since2022.csv", index=False)  # CSV 파일로 저장

In [18]:
df['journal'] = "JAIS: Journal of the Association for Information Systems"
df.to_csv("JAIS_papers_since2022.csv", index=False)